# Natural Language processing

-Udyot Kumar

Here I have a datsaet which contains 1000 reviews on a restaurant. All reviews are assigned 1(liked) and 0(disliked). The main aim of this project is to create a supervised machine learning model which can classify future reviews into liked and disliked categories. So to start the project I used packages 're' and 'nltk' to clean the reviews and converted them into a usable form. I will explain how I have cleaned the dataset along with the code. After that I applied Naive Bayes, K nearest neighbours, Support vector machines and random forests on the dataset and then have compared the classification rate for each of those 4 models on the test dataset which is 25% of total dataset.

From this project, I have learned how to work on natural language processing and this is my first attempt on it. I will keep on posting more projects on NLP very soon.

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

### Importing the dataset,
Quoting=3 ignores the double quotes in the dataset

In [2]:
dataset= pd.read_csv('C:\Machine Learning A-Z\Natural Language Processing\Natural_Language_Processing\Restaurant_Reviews.tsv',
                     delimiter= '\t',
                     quoting= 3)

#### If you want to learn how to install a package then you can use the below code:

In [4]:
import pip    
def install(package):
   pip.main(['install', package])

In [6]:
install('nltk')

  Running setup.py install for nltk: started
    Running setup.py install for nltk: finished with status 'done'


You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


### Importing nltk and re (regular expressions)

In [7]:
import re
import nltk

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\I854950\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Below you can see the first 10 reviews in my dataset.

In [11]:
dataset.head(10)

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
5,Now I am getting angry and I want my damn pho.,0
6,Honeslty it didn't taste THAT fresh.),0
7,The potatoes were like rubber and you could te...,0
8,The fries were great too.,1
9,A great touch.,1


In [12]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

### Cleaning methodology:

1. You can see texts like "...", ".", ",", ")" etc. So we have taken care of them by using this line: review= re.sub('[^a-zA-Z]',' ', dataset['Review'][i]) This line of code takes only the character between a-z and A-Z in the reviews and removes everything else from the reviews.
2. Now there is another thing called as stop words (http://www.ranks.nl/stopwords) we have to remove the stopwords too from our reviews as stopwords don't tell us the exact essence of the review
3. After removing stopwords, we take stem of each word i.e. running--> run, laughed-->laugh So this converts all the words into their step form.

Thus we have reviews containing only english alphabets, without any stopwords, and all the words in their Stem form

In [25]:
corpus= []
for i in range(0, 1000):
    #only taking alphabets between a-z and A-Z
    review= re.sub('[^a-zA-Z]',' ', dataset['Review'][i]) 
    #converting evry character to lower case
    review= review.lower() 
    review=review.split() 
    ps = PorterStemmer()
    #taking stem or base of the words excluding stopwords from the review
    review= [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    #adding ' ' (space) in the review
    review= ' '.join(review)
    corpus.append(review)

here you can see just the list of reviews with all stem words and without stopwords 

In [38]:
corpus[0:5]

[u'wow love place',
 'crust good',
 u'tasti textur nasti',
 u'stop late may bank holiday rick steve recommend love',
 u'select menu great price']

### Creating bag of words model 

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features= 1500)  #sparse matrix- contains a lot of 0
X= cv.fit_transform(corpus).toarray()
y= dataset.iloc[:,1].values

Splitting the dataset into the Training set and Test set

In [18]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

C:\Users\I854950\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Fitting Naive Bayes to the Training set

In [19]:
from sklearn.naive_bayes import GaussianNB
classifierNB = GaussianNB()
classifierNB.fit(X_train, y_train)

GaussianNB(priors=None)

Predicting the Test set results for Naive Bayes

In [20]:
y_pred_NB = classifierNB.predict(X_test)

Making the Confusion Matrix

In [40]:
from sklearn.metrics import confusion_matrix
cm_NB = confusion_matrix(y_test, y_pred_NB)

In [41]:
cm_NB

array([[ 67,  50],
       [ 20, 113]])

#### Misclassification rate for Naive Bayes=0.28

### Fitting K-NN to the Training set, 
Here we are taking value of K as 5. You can change it according to your use case. P=2 means we are using Standard euclidean distance

In [42]:
from sklearn.neighbors import KNeighborsClassifier
classifierKNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifierKNN.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

Predicting the Test set results for KNN

In [44]:
y_pred_KNN = classifierKNN.predict(X_test)

Making the Confusion Matrix

In [45]:
from sklearn.metrics import confusion_matrix
cm_KNN = confusion_matrix(y_test, y_pred_KNN)

In [46]:
cm_KNN

array([[95, 22],
       [72, 61]])

#### Misclassification rate for KNN is = 0.376

### Fitting Support vector machine to the training set

We can change the parameter kernel to poly, rbf, sigmoid

In [64]:
from sklearn.svm import SVC
classifierSVM = SVC(kernel = 'linear', random_state = 0)
classifierSVM.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

Predicting the Test set results

In [65]:
y_pred_SVM = classifierSVM.predict(X_test)

Making the Confusion Matrix

In [66]:
from sklearn.metrics import confusion_matrix
cm_SVM = confusion_matrix(y_test, y_pred_SVM)

In [67]:
cm_SVM

array([[94, 23],
       [49, 84]])

#### Misclassification rate for SVM is = 0.288

### Fitting Random Forest to the training set

In [78]:
from sklearn.ensemble import RandomForestClassifier
classifierRF = RandomForestClassifier(n_estimators = 1000, criterion = 'entropy', random_state = 0)
classifierRF.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

Predicting the Test set results

In [79]:
y_pred_RF = classifierRF.predict(X_test)

Making the Confusion Matrix

In [80]:
cm_RF = confusion_matrix(y_test, y_pred_RF)

In [81]:
cm_RF

array([[104,  13],
       [ 64,  69]])

#### Misclassification rate for Random Forest is 0.308

## So after creating these 4 models we have come up to a conclusion that: Naive bayes is working the best here, followed by support vector machines, random forests and the last one is K-NN